In [2]:
import numpy as np
import cv2
import glob
import json
import dlib
from imutils import face_utils
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras_vggface.vggface import VGGFace
import face_recognition
import math
import face_alignment

Using TensorFlow backend.


In [3]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)

# with open('uuid_lengths_128.json') as json_file:
#     uuid_lengths_128 = json.load(json_file)

participants = glob.glob('dataset/*')
# participants = sorted(participants)
    
DATASET_SIZE = len(keys_all)

In [5]:
fa2D = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device='cpu')

In [6]:
fa3D = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False, device='cpu')

In [7]:
# Loop over each participant
uuid_lengths_FAN = {}
fail_counter = 0
for j in range(1):
    
    # Extract the UUID
    uuid_excluded = participants[j].split('/')[1]

    # Loop over the dataset to remove the examples associated with this participant
    indices_excluded = []
    keys_excluded = []
    for i in range(DATASET_SIZE):
        key = keys_all[i]
        uuid = key.split('/')[0]
        if(uuid == uuid_excluded):

            indices_excluded.append(i)
            keys_excluded.append(key)
    
    # Construct the validation dataset consisting of examples of a single participant
    # Evaluation of the classifiers will be done on this dataset
    image_points = np.zeros((len(keys_excluded), 68, 2))
    model_points = np.zeros((len(keys_excluded), 68, 3))
    labels = np.zeros(len(keys_excluded))
    
    failed_indices = []
    for i in range(len(keys_excluded)):
        key = keys_excluded[i]
        label = key.split('/')[1]

        im = cv2.imread('dataset/' + key)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im_points = np.asarray(fa2D.get_landmarks(im))
        mod_points = np.asarray(fa3D.get_landmarks(im))
        if(im_points.shape != (68, 2)):
            max_dist = 0
            max_idx = 0
            i = 0
            while(i < im_points.shape[0]):
                if(abs(im_points[i, 8, 1] - im_points[i, 27, 1]) > max_dist):
                    max_dist = abs(im_points[i, 8, 1] - im_points[i, 27, 1])
                    max_idx = i
                i += 1
            im_points = im_points[max_idx, :]
            mod_points = mod_points[max_idx, :]
            
        image_points[i, :] = im_points.reshape(68, 2)
        model_points[i, :] = mod_points.reshape(68, 3)
        if(label=='positive'):
            labels[i] = 1
        else:
            labels[i] = 0
#         for point in image_points[0, :]:
#             cv2.circle(im, (int(point[0]), int(point[1])), 3, (0, 0, 255), -1)
#         cv2.imshow('Test', im)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
    
    uuid_lengths_FAN[uuid_excluded] = labels.shape[0]
    np.save('FAN_landmarks_kitsios/image_points_' + uuid_excluded , image_points)
    np.save('FAN_landmarks_kitsios/model_points_' + uuid_excluded, model_points)
    np.save('FAN_landmarks_kitsios/labels_' + uuid_excluded, labels)

In [8]:
print(np.sum(labels))

2.0
